In [43]:
import numpy as np
import pandas as pd
from preprocess.load_data.data_loader import load_hotel_reserve

# 第3章 集約
## 3-1 データ数、種類数の算出

In [51]:
customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


In [25]:
result = reserve_tb.groupby(['hotel_id']).agg({'reserve_id': 'count', 'customer_id': 'nunique'})
result.head()

,reserve_id,customer_id
hotel_id,,
h_1,10,10
h_10,3,3
h_100,20,19
h_101,17,17
h_102,13,13


In [26]:
result.reset_index(inplace=True)
result.columns = ['hotel_id', 'rsv_cnt', 'cus_cnt']
result.head()

,hotel_id,rsv_cnt,cus_cnt
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13


## 3-2 合計値の算出

In [32]:
result = reserve_tb.groupby(['hotel_id', 'people_num']).agg({'total_price': 'sum'})
result.reset_index(inplace=True)
result.columns = ['hotel_id', 'people_num', 'price_sum']
result.head()

,hotel_id,people_num,price_sum
0,h_1,1,156600
1,h_1,2,156600
2,h_1,3,391500
3,h_1,4,417600
4,h_10,1,11200


## 3-3 極値、代表値の算出
### 代表値

In [45]:
result = reserve_tb.groupby(['hotel_id']).agg({'total_price': ['max', 'min', 'mean', lambda x: np.percentile(x, q=20)]})
result.reset_index(inplace=True)
# result.columns = ['hotel_id', 'people_num', 'price_sum']
result.head()

hotel_id total_price                               
                   max    min           mean <lambda>
0      h_1      208800  26100  112230.000000    73080
1     h_10       67200  11200   42933.333333    26880
2    h_100       57600   4800   27600.000000     9600
3    h_101      168000  14000   75764.705882    30800
4    h_102       72000  12000   32769.230769    18000

## 3-4 ばらつき具合の算出

In [49]:
result = reserve_tb.groupby(['hotel_id']).agg({'total_price': ['var', 'std']}) 
result.reset_index(inplace=True)
# result.columns = ['hotel_id', 'people_num', 'price_sum']
result.fillna(0, inplace=True)
result.head()

hotel_id   total_price              
                     var           std
0      h_1  3.186549e+09  56449.526127
1     h_10  8.258133e+08  28736.968061
2    h_100  3.198316e+08  17883.835689
3    h_101  2.402441e+09  49014.703676
4    h_102  3.576923e+08  18912.755159

## 3-5 最頻値の算出

In [52]:
reserve_tb['total_price'].round(-3).mode()

0    10000
1    20000
2    40000
dtype: int64

## 3-6 順位の算出
### 時系列に番号を付与

In [55]:
reserve_tb['reserve_datetime'] = pd.to_datetime(reserve_tb['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')
reserve_tb['log_no'] = reserve_tb.groupby(['customer_id'])['reserve_datetime'].rank(ascending=True, method='first')
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,log_no
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,1.0
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,2.0
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,3.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,4.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,5.0


### ランキング

In [74]:
rsv_cnt_tb = reserve_tb.groupby('hotel_id').size().reset_index()
rsv_cnt_tb.columns = ['hotel_id', 'rsv_cnt']
rsv_cnt_tb.head()

,hotel_id,rsv_cnt
0,h_1,10
1,h_10,3
2,h_100,20
3,h_101,17
4,h_102,13


In [75]:
rsv_cnt_tb['rsv_cnt_rank'] = rsv_cnt_tb['rsv_cnt'].rank(ascending=False, method='min')
rsv_cnt_tb.drop('rsv_cnt', axis=1, inplace=True)
rsv_cnt_tb.head()

,hotel_id,rsv_cnt_rank
0,h_1,235.0
1,h_10,300.0
2,h_100,12.0
3,h_101,43.0
4,h_102,139.0
